# PIP model 🐾 🤖
In this notebook I import and play with my PIP model, designed in blender

In [1]:
!pip install pybullet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 MB 5.0 MB/s eta 0:00:00


In [2]:
import pybullet as p
import pybullet_data
import numpy as np
import os

# Connect to PyBullet
p.connect(p.DIRECT)

# Set additional search path for PyBullet
p.setAdditionalSearchPath(pybullet_data.getDataPath())

# Load a plane for the simulation
p.loadURDF("plane.urdf")


0

In [3]:
!git clone https://github.com/Gaianeve/Real_Mega_Fufi.git

Cloning into 'Real_Mega_Fufi'...
remote: Enumerating objects: 108, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 108 (delta 41), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (108/108), 466.64 KiB | 3.67 MiB/s, done.
Resolving deltas: 100% (41/41), done.


## Loading model 📡 🛰

In [4]:
%cd Real_Mega_Fufi/Robot/PIPPA

/content/Real_Mega_Fufi/Robot/PIPPA


In [6]:
PIPPA_id = p.loadURDF("PIPPA.urdf")


## Useful info and tools 🎮 📚 🏮
Print name and index of the joints and links. Link indexes are estabilished based on the links hierarchy, that's why `'Bottom_straight_link'` act as the base, and therefor has index `-1`. The other links are all indexed according to the order the are defined with in the urdf file.

              Practical table not to be wrong
---
---

Nome del Link	  🔗 📛   -------->       Indice 🔢 🥦

---

* Bottom_straight_link -----> -1
* Bottom_Link_1	-----> 0
* Bottom_Link_2	-----> 1
* Bottom_Link_3	-----> 2
* Colonna_Link_1	-----> 3
* Colonna_Link_2	-----> 4
* Colonna_Link_3	-----> 5
* Cover_Link1	-----> 6
* Cover_Link_2	-----> 7
* Cover_Link_3	-----> 8
* Elastic_Joint_Link_1 ----->	9
* Elastic_Joint_link_2 ----->	10
* Elastic_Joint_link_3 ----->	11
* Gamba_Link_1	-----> 12
* Gamba_Link_2	-----> 13
* Gamba_Link_3	-----> 14
* Top_Link_1	-----> 15
* Top_Link_2	-----> 16
* Top_Link_3	-----> 17


In [7]:
# Get the number of joints
num_joints = p.getNumJoints(PIPPA_id)
print(f"Number of joints: {num_joints}")

# Get information about each joint
for joint_index in range(num_joints):
    joint_info = p.getJointInfo(PIPPA_id, joint_index)
    joint_name = joint_info[1].decode('UTF-8')  # Decode bytes to string
    joint_type = joint_info[2]  # Joint type (e.g., revolute, spherical)

    print(f"Joint Index: {joint_index}, Name: {joint_name}, Type: {joint_type}")

Number of joints: 18
Joint Index: 0, Name: Bottom_straight_to_1, Type: 4
Joint Index: 1, Name: Bottom_to_Column_1, Type: 4
Joint Index: 2, Name: elastic_to_column_1, Type: 4
Joint Index: 3, Name: Leg_to_elastic_1, Type: 4
Joint Index: 4, Name: Cover_to_leg_1, Type: 4
Joint Index: 5, Name: Cover_to_top_1, Type: 4
Joint Index: 6, Name: Bottom_straight_to__3, Type: 4
Joint Index: 7, Name: Bottom_to_column3, Type: 4
Joint Index: 8, Name: elastic_to_column_3, Type: 4
Joint Index: 9, Name: Leg_to_elastic_3, Type: 4
Joint Index: 10, Name: Cover_to_leg_3, Type: 4
Joint Index: 11, Name: Cover_to_top_3, Type: 4
Joint Index: 12, Name: Bottom_stright_to_2, Type: 4
Joint Index: 13, Name: Bottom_to_column_2, Type: 4
Joint Index: 14, Name: elastic_to_column_2, Type: 4
Joint Index: 15, Name: Leg_to_elastic_2, Type: 4
Joint Index: 16, Name: Cover_to_leg_2, Type: 4
Joint Index: 17, Name: Cover_to_top_2, Type: 4


In [10]:
# Indexes of the links in the urdf file. This function basically gives the definition order.
import xml.etree.ElementTree as ET

# Load the URDF file
urdf_file_path = 'PIPPA.urdf'

# Parse the URDF file
tree = ET.parse(urdf_file_path)
root = tree.getroot()

# Find all links in the URDF file and print their names with indices
links = []
for i, link in enumerate(root.findall('link')):
    link_name = link.attrib.get('name', f'Unnamed_{i}')
    links.append((i, link_name))

# Print link indices and names
for index, name in links:
    print(f"Link index: {index}, Link name: {name}")


Link index: 0, Link name: Bottom_Link_1
Link index: 1, Link name: Bottom_Link_2
Link index: 2, Link name: Bottom_Link_3
Link index: 3, Link name: Bottom_straight_link
Link index: 4, Link name: Colonna_Link_1
Link index: 5, Link name: Colonna_Link_2
Link index: 6, Link name: Colonna_Link_3
Link index: 7, Link name: Cover_Link1
Link index: 8, Link name: Cover_Link_2
Link index: 9, Link name: Cover_Link_3
Link index: 10, Link name: Elastic_Joint_Link_1
Link index: 11, Link name: Elastic_Joint_link_2
Link index: 12, Link name: Elastic_Joint_link_3
Link index: 13, Link name: Gamba_Link_1
Link index: 14, Link name: Gamba_Link_2
Link index: 15, Link name: Gamba_Link_3
Link index: 16, Link name: Top_Link_1
Link index: 17, Link name: Top_Link_2
Link index: 18, Link name: Top_Link_3


## Changing material of the links ⛵ 🦚
Assign material trough the properties by changing dinamics. Everything about elastic joint Young modulus is being dealt with in the urdf file.

Basically, the joint is modelized as a *neo hookean* soft body  ♨ 🗻. Again, see the guide or my notes for the details.

In [12]:
# AL 6082 (Alluminio)
lateralFriction_Al = 0.3
restitution_Al = 0.2
rollingFriction_Al = 0.001
spinningFriction_Al = 0.01
linearDamping_Al = 0.04
angularDamping_Al = 0.1
contactStiffness_Al = 3e6
contactDamping_Al = 800

# AISI 304 (Acciaio Inox)
lateralFriction_AISI = 0.4
restitution_AISI = 0.1
rollingFriction_AISI = 0.001
spinningFriction_AISI= 0.05
linearDamping_AISI = 0.03
angularDamping_AISI = 0.1
contactStiffness_AISI = 8e6
contactDamping_AISI = 1500

# Acciaio Maraging (Maragià per gli amici)
lateralFriction_Mar = 0.3
restitution_Mar = 0.2
rollingFriction_Mar = 0.001
spinningFriction_Mar = 0.02
linearDamping_Mar = 0.04
angularDamping_Mar = 0.15
contactStiffness_Mar = 3e6
contactDamping_Mar = 800

### Physics parameters explanation 🎈 🔮

1.  `lateralFriction=0.9` 🐑

  *   **Descrizione**: Rappresenta il coefficiente di attrito laterale tra il link e la superficie su cui entra in contatto.
  *   **Effetto:**  Più alto è il valore, maggiore sarà l'attrito tra il link e la superficie, il che significa che il link resisterà di più allo scivolamento lungo la superficie. Un valore di 1.0 o vicino a esso significa un attrito alto, mentre un valore basso (come 0.1) indica poco attrito, e il link scivolerà più facilmente.

  *   **Esempio:** Materiali ruvidi o che non scivolano facilmente, come gomma o superfici ruvide, hanno un coefficiente di attrito più alto. Materiali come ghiaccio o superfici lisce e scivolose avranno un coefficiente basso.

*Caso Acciaio*: Il valore 0.9 è abbastanza alto, riflettendo l'alto attrito dell'acciaio maraging a contatto con superfici ruvide o altri metalli.


2.  `restitution=0.1` 🐬

  *   **Descrizione**: Rappresenta il coefficiente di restituzione, che determina quanto "rimbalza" un oggetto dopo un impatto.

  *   **Effetto:**  Un valore di 1.0 significa che l'oggetto rimbalza completamente (energia conservata), mentre un valore di 0.0 significa che non c'è rimbalzo (l'energia viene completamente assorbita).

  *   **Esempio:** Materiali elastici come la gomma hanno un alto coefficiente di restituzione (vicino a 1.0), mentre materiali rigidi e non elastici come il piombo o l'acciaio hanno un coefficiente basso.

*Caso Acciaio*: L'acciaio ha un basso coefficiente di restituzione (intorno a 0.1), il che significa che non rimbalza molto dopo un impatto, riflettendo la sua natura rigida e non elastica.


3.  `contactStiffness=1e6` 🐨

  *   **Descrizione**: Rappresenta la rigidità del contatto, o quanto è resistente il materiale alla compressione quando viene a contatto con un'altra superficie.

  *   **Effetto:**  Un valore più alto significa che il materiale è molto rigido e si deformerà pochissimo durante un contatto (simulando l'effetto di un modulo di Young alto), mentre un valore basso rappresenta materiali più morbidi che si deformano più facilmente.

  *   **Esempio:** Materiali come il gommapiuma hanno una bassa rigidità del contatto, mentre materiali molto rigidi come acciaio o ceramica hanno un'alta rigidità.

*Caso Acciaio*: Un valore come 1e6 simula una rigidezza molto alta, tipica dell'acciaio maraging, che si deforma molto poco sotto pressione.


4.  `contactDamping=1e3` 🦘
  
  *   **Descrizione**: Rappresenta lo smorzamento del contatto, che controlla quanto velocemente l'energia del contatto viene dissipata nel tempo.

  *   **Effetto:** Un valore più alto significa che il contatto perde energia rapidamente, smorzando le vibrazioni o il rimbalzo. Un valore basso implica che il sistema è più elastico e manterrà energia più a lungo.

  *   **Esempio:** Un materiale molto elastico, come una molla ben tesa, ha uno smorzamento basso, il che significa che rimbalza e vibra a lungo. Un materiale con alto smorzamento, come l'acciaio, disperde rapidamente energia durante un impatto, quindi non rimbalza molto e smorza le vibrazioni.

*Caso Acciaio*: Il valore 1e3 rappresenta uno smorzamento moderato, coerente con il comportamento dell'acciaio maraging, che dissipa energia abbastanza rapidamente senza rimbalzi significativi.


❕📔🖊 *Note* : *For more detailon the parameters see the parameters see my notes or pybullet Quickstart guide, everything's in there*.

❗ Not all parameters are important when simulating, so caution with that choise. **To be decided with max.**

In [14]:
link_indices_Al = [-1, 0, 1, 2, 6, 7, 8, 12, 13, 14]
link_indices_AISI = [3, 4, 5, 15, 16, 17]
link_indices_Mar = [9, 10, 11]

# Al 6082
for link in link_indices_Al:
  p.changeDynamics(PIPPA_id, link, lateralFriction=lateralFriction_Al,\
                   restitution=restitution_Al, \
                   spinningFriction= spinningFriction_Al, rollingFriction= rollingFriction_Al,\
                   linearDamping=linearDamping_Al, angularDamping=angularDamping_Al,\
                   contactStiffness = contactStiffness_Al,contactDamping = contactDamping_Al)
# AISI 304
for link in link_indices_AISI:
  p.changeDynamics(PIPPA_id, link, lateralFriction=lateralFriction_AISI,\
                   restitution=restitution_AISI, \
                   spinningFriction= spinningFriction_AISI, rollingFriction= rollingFriction_AISI,\
                   linearDamping=linearDamping_AISI, angularDamping=angularDamping_AISI,\
                   contactStiffness = contactStiffness_AISI, contactDamping = contactDamping_AISI)
# Maragià
for link in link_indices_AISI:
  p.changeDynamics(PIPPA_id, link, lateralFriction=lateralFriction_Mar,\
                   restitution=restitution_Mar, \
                   spinningFriction= spinningFriction_Mar, rollingFriction= rollingFriction_Mar,\
                   linearDamping=linearDamping_Mar, angularDamping=angularDamping_Mar,\
                   contactStiffness = contactStiffness_Mar, contactDamping = contactDamping_Mar)

## Start simulation 🚀 🪖
Adding gravity 🍎 ⬇ to the system and starting the physics simulation 🏃 🔥 🛕 ⛩ ⛪ .


In [16]:
# Set gravity for the simulation
p.setGravity(0, 0, -9.81)

p.resetSimulation()
force = np.array([1,1,2])
force_position = np.array([0,0,0])

initial_position_base, initial_orientation_base = p.getBasePositionAndOrientation(PIPPA_id)
initial_position_leg, initial_orientation_leg = p.getLinkState(PIPPA_id, joint_index)[:2]

print("Posizione iniziale del giunto 14:", initial_position_leg)
print("Orientamento iniziale del giunto 14:", initial_orientation_leg)
print("Posizione iniziale Base:", initial_position_base)
print("Orientamento iniziale Base:", initial_orientation_base)

p.applyExternalForce(
    objectUniqueId=PIPPA_id,linkIndex= -1,forceObj=force,posObj=force_position,flags=p.WORLD_FRAME)

for _ in range(1):
    p.stepSimulation()  # Aggiorna la simulazione
    time.sleep(1./240.)  # Ritardo per simulare 240 Hz

final_position_base, final_orientation_base = p.getBasePositionAndOrientation(PIPPA_id)
final_position_leg, final_orientation_leg = p.getLinkState(PIPPA_id, joint_index)[:2]

print("Posizione finale del giunto 14:", final_position_leg)
print("Orientamento finale del giunto 14:", final_orientation_leg)

print("Posizione finale Base:", final_position_base)
print("Orientamento finale Base:", final_orientation_base)

error: GetBasePositionAndOrientation failed.